<a href="https://colab.research.google.com/github/junsung12/find_fraud/blob/master/%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df2 = pd.read_csv('/content/any_gen.csv')

In [ ]:
df2.head(1)

,title,genre,summary
0,은혼 1기,"판타지,액션","에도 시대 말기, 하늘에서 내려온 ‘천인’의 침략으로 전쟁이 발생한다. 막부들은 결..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


In [ ]:
df2['summary'].isnull().values.any()

False

In [ ]:
tfidf_matrix = tfidf.fit_transform(df2['summary'])
tfidf_matrix.shape

(2230, 40516)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

cosine_sim

array([[1.        , 0.00575962, 0.00550179, ..., 0.00609494, 0.        ,
        0.01585575],
       [0.00575962, 1.        , 0.026652  , ..., 0.00377214, 0.        ,
        0.00673373],
       [0.00550179, 0.026652  , 1.        , ..., 0.00360329, 0.        ,
        0.00641884],
       ...,
       [0.00609494, 0.00377214, 0.00360329, ..., 1.        , 0.        ,
        0.01072022],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01585575, 0.00673373, 0.00641884, ..., 0.01072022, 0.        ,
        1.        ]])

In [ ]:
cosine_sim.shape

(2230, 2230)

In [ ]:
indices = pd.Series(df2.index, index=df2['title'])
indices

title
은혼 1기                        0
주술회전 1기 part 1               1
귀멸의 칼날                       2
전생했더니 슬라임이었던 건에 대하여 1기       3
꿈빛 파티시엘 리마스터                 4
                          ... 
집이 없어                     2225
케어베어 : 나도 상 받고 싶어         2226
죠죠의 기묘한 모험 시즌4: 황금의 바람    2227
벅스 프렌즈                    2228
MONSTER                   2229
Length: 2230, dtype: int64

In [ ]:
# 양화 제목 입력 받으면 유사도 -> 가장 유사도 높은 top10
def get_recommendations(title,cosine_sim = cosine_sim):
    idx= indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores,key=lambda x:x[1],reverse=True)

    sim_scores = sim_scores[1:11]

    test_any_indices = [i[0] for i in sim_scores]

    return df2['title'].iloc[test_any_indices]




In [ ]:
test_idx = indices['귀멸의 칼날']
test_idx

2

In [ ]:
cosine_sim[2]

array([0.00550179, 0.026652  , 1.        , ..., 0.00360329, 0.        ,
       0.00641884])

In [ ]:
test_sim_scores=list(enumerate(cosine_sim[3]))  # 코사인 유사도 매트리스 (cosine_sim)에서 idx에 해당하는 데이터를(idx, 유사도)형태로 얻기

In [ ]:
test_sim_scores = sorted(test_sim_scores,key=lambda x:x[1],reverse=True)
test_sim_scores[1:11] #  자기 제외 한 10개 추천

[(539, 0.9999999999999999),
 (184, 0.08555127232022884),
 (177, 0.05875153923193569),
 (2070, 0.05130956075460931),
 (1398, 0.05130709476241843),
 (400, 0.05103058761526067),
 (319, 0.048002141791299496),
 (540, 0.04782600580669891),
 (288, 0.04780530167049746),
 (2198, 0.04627604806509695)]

In [ ]:
# 추천 영화 목록 10개 인덱스 정보
test_any_indices = [i[0] for i in test_sim_scores[1:11]]
test_any_indices

[539, 184, 177, 2070, 1398, 400, 319, 540, 288, 2198]

In [ ]:
# 인덱스 정보를 통해 영화 제목 추출
df2['title'].iloc[test_any_indices]

539                      전생했더니 슬라임이었던 건에 대하여 OAD
184               전생했더니 슬라임이었던 건에 대하여 : 전생슬라임 일기
177                                    나만이 없는 거리
2070                                  로티프렌즈 뚝딱미술
1398                               헬로 카봇 시즌 13 젬
400                                      이세계 콰르텟
319                       단간론파 : 희망의 학교와 절망의 고교생
540                                    이세계 콰르텟 2
288     전생현자의 이세계 라이프 ~두 번째 직업을 얻고 세계 최강이 되었습니다~
2198                           실바니안 패밀리: 프레야의 선물
Name: title, dtype: object

# 원하는 영화 넣어서 추천 받기

In [ ]:
df2['title'][:15]

0                      은혼 1기
1             주술회전 1기 part 1
2                     귀멸의 칼날
3     전생했더니 슬라임이었던 건에 대하여 1기
4               꿈빛 파티시엘 리마스터
5           귀멸의 칼날 : 환락의 거리편
6                   하이큐!! 1기
7             주술회전 1기 part 2
8                   하이큐!! 2기
9          SPY×FAMILY part 1
10           귀멸의 칼날 : 도공 마을편
11                 데스노트 리마스터
12                   장송의 프리렌
13                 진격의 거인 1기
14                    최애의 아이
Name: title, dtype: object

## 귀멸의 칼날과 비슷한 애니 top10

In [ ]:
get_recommendations('귀멸의 칼날')

106          귀멸의 칼날 : 무한열차편
708         귀멸의 칼날 : 나타구모산편
970          귀멸의 칼날 : 아사쿠사편
170                우라미치 선생님
1949              반도에 살어리랏다
9         SPY×FAMILY part 1
23        SPY×FAMILY part 2
86      SPY×FAMILY Season 2
243                    닥터스톤
1095         귀멸의 칼날 : 장구저택편
Name: title, dtype: object